In [1]:
import re
import pandas as pd
import numpy as np
import PyPDF2

### NaturDoc - TL BL WT 22-23

# Data transformation:

Datasets we have explored:
* not useable by themselves
* not matching visions of team for information that the app should provide

So we delve deeper into the datasets and try to manipulate them to extract the information we need, in hopes of creating a final enriched dataset at the end.

## Imports:

### Duke:

In [2]:
duke_df = pd.read_csv("../data/DUKE/ETHNOBOT.csv")

### KEW:

The two KEW datasets with the most promising geographic data.

_Note: both of these csv files were too large to be pushed to GitHub. We will modify them in this notebook to export to csv with a reduced file size and make them more useable for us as well, as some code can be very time-consuming to run with the way the files are set up right now._

In [5]:
wcvp_dis_df = pd.read_csv("../data/WCVP/.wcvp_distribution.csv", sep="|")
wcvp_names_df = pd.read_csv("../data/WCVP/.wcvp_names.csv", sep="|")

/tmp/ipykernel_12162/3174618310.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  wcvp_names_df = pd.read_csv("../data/WCVP/.wcvp_names.csv", sep="|")


### WHO Monograph 1:

In [ ]:
who_01_pdf = open('../data/WHO/monograph_01.pdf', 'rb')

who_01_reader = PyPDF2.PdfReader(who_01_pdf)

who_01_pages = list()

pattern_fi = re.compile("ﬁ")
pattern_fl = re.compile("ﬂ")
pattern_minus = re.compile("–")
pattern_brackets = re.compile("\([\s0-9,-]+\)")
pattern_spacedot = re.compile(" \.")
pattern_spacecomma = re.compile(" ,")
pattern_spacequotesingle = re.compile(" ’")
pattern_spacequotedouble = re.compile(" ”")
pattern_doublespace = re.compile("\s\s+")
pattern_dashspace = re.compile("-\s+")
  
for page in range(len(who_01_reader.pages)):
    page_obj = who_01_reader.pages[page]
    text = page_obj.extract_text()
    text = re.sub(pattern_fi, "fi", text)
    text = re.sub(pattern_fl, "fl", text)
    text = re.sub(pattern_minus, "-", text)
    text = re.sub(pattern_brackets, "", text)
    text = re.sub(pattern_spacedot, ".", text)
    text = re.sub(pattern_spacecomma, ",", text)
    text = re.sub(pattern_spacequotesingle, "’", text)
    text = re.sub(pattern_spacequotedouble, "”", text)
    text = re.sub(pattern_doublespace, " ", text)
    text = re.sub(pattern_dashspace, "", text)
    lines = text.split("\n")
    who_01_pages.append(lines)
    
who_01_pdf.close()

In [ ]:
p_01_50 = who_01_pages[7 : 57]
p_51_58 = who_01_pages[75 : 83]
p_59_60 = who_01_pages[57 : 59]
p_61_66 = who_01_pages[69 : 75]
p_67_76 = who_01_pages[59 : 69]
p_77_end = who_01_pages[83 : ]

who_01_pages_corrected = p_01_50 + p_51_58 + p_59_60 + p_61_66 + p_67_76 + p_77_end

## Transforming the DUKE dataset:

The original Dataset of the _Dr. Duke's Phytochemical and Ethnobotanical Databases_ contains a lot of information that we do not need for the scope of our app. Currently the dataset pairs unique herbs with a single "activity", linked to relevant research and other data, but for our needs it makes sense to pivot the dataframe by each herb, aggregating their data all in a single row.

As np.NaN values cause issues when joining the aggregated values subsequently, all np.NaN values will first be replaced with a custom nan string value.

In [3]:
target_cols = ["TAXON", "ACTIVITY", "CNAME", "FAMILY", "GENUS", "SPECIES"]
mod_duke_df = duke_df[target_cols].replace(np.NaN, "nan").pivot_table(index='TAXON', aggfunc=pd.unique)
mod_duke_df.head()

,ACTIVITY,CNAME,FAMILY,GENUS,SPECIES
TAXON,,,,,
Abelmoschus esculentus,"[Abortifacient, Antidote, Boil, Burn, Catarrh,...","[nan, Huang Shu K'Uei, Bamia, Molondron, Quiab...",[Malvaceae],[Abelmoschus],[esculentus]
Abelmoschus manihot,"[Antitussive, Boil, Cancer, Cancer(Stomach), C...","[Tororo-Aoi, nan, Kastuli]",[Malvaceae],[Abelmoschus],[manihot]
Abelmoschus moschatus,"[Aphrodisiac, Asthma, Bite(Snake), Boil, Cance...","[Musk Mallow, nan, Moskus, Muskus, Mushk Dana,...",[Malvaceae],[Abelmoschus],[moschatus]
Abies alba,"[Bronchitis, Bruise, Calculus, Catarrh, Cough,...","[Abeto, Edeltanne, Beyaz Koknar, Silver Fir, A...",[Pinaceae],[Abies],[alba]
Abies balsamea,"[Burn, Cancer, Cold, Cough, Heart, Masticatory...","[nan, Balsam,Canada, Balsam Fir, Fir]",[Pinaceae],[Abies],[balsamea]


While multiple values are stored in a list for now, exporting and later re-importing this dataframe as a csv causes issues (the list will instead be treated as a string). This will be further explored in the "Pivoting" as well as the "Comparing output of these new csv files" subchapters of the section transforming the KEW datasets.

The following code instead joins the multiples values in a comma separated string:

In [ ]:
mod_duke_df["ACTIVITY"] = mod_duke_df["ACTIVITY"].str.join(",")
mod_duke_df["CNAME"] = mod_duke_df["CNAME"].str.join(",")
mod_duke_df["FAMILY"] = mod_duke_df["FAMILY"].str.join(",")
mod_duke_df["GENUS"] = mod_duke_df["GENUS"].str.join(",")
mod_duke_df["SPECIES"] = mod_duke_df["SPECIES"].str.join(",")
mod_duke_df.head()

,ACTIVITY,CNAME,FAMILY,GENUS,SPECIES
TAXON,,,,,
Abelmoschus esculentus,"Abortifacient,Antidote,Boil,Burn,Catarrh,Coffe...","nan,Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,...",Malvaceae,Abelmoschus,esculentus
Abelmoschus manihot,"Antitussive,Boil,Cancer,Cancer(Stomach),Catarr...","Tororo-Aoi,nan,Kastuli",Malvaceae,Abelmoschus,manihot
Abelmoschus moschatus,"Aphrodisiac,Asthma,Bite(Snake),Boil,Cancer,Car...","Musk Mallow,nan,Moskus,Muskus,Mushk Dana,Amber...",Malvaceae,Abelmoschus,moschatus
Abies alba,"Bronchitis,Bruise,Calculus,Catarrh,Cough,Diure...","Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ...",Pinaceae,Abies,alba
Abies balsamea,"Burn,Cancer,Cold,Cough,Heart,Masticatory,Sore,...","nan,Balsam,Canada,Balsam Fir,Fir",Pinaceae,Abies,balsamea


_Note: later, it is important to remember to remove the null values when they appear as part of a list._

The dataframe is now indexed by the unique taxonomic name. If necessary, a regular serial index can be created by executing reset_index(). This is helpful in that "TAXON" will then become a normal column again and can therefore be targeted with .loc.

_Edit: when exporting the dataframe to a new CSV, another serial index gets added upon reading it again later so this is not necessary._

In [ ]:
mod_duke_df = mod_duke_df.reset_index()
mod_duke_df.head()

,TAXON,ACTIVITY,CNAME,FAMILY,GENUS,SPECIES
0,Abelmoschus esculentus,"Abortifacient,Antidote,Boil,Burn,Catarrh,Coffe...","nan,Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,...",Malvaceae,Abelmoschus,esculentus
1,Abelmoschus manihot,"Antitussive,Boil,Cancer,Cancer(Stomach),Catarr...","Tororo-Aoi,nan,Kastuli",Malvaceae,Abelmoschus,manihot
2,Abelmoschus moschatus,"Aphrodisiac,Asthma,Bite(Snake),Boil,Cancer,Car...","Musk Mallow,nan,Moskus,Muskus,Mushk Dana,Amber...",Malvaceae,Abelmoschus,moschatus
3,Abies alba,"Bronchitis,Bruise,Calculus,Catarrh,Cough,Diure...","Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ...",Pinaceae,Abies,alba
4,Abies balsamea,"Burn,Cancer,Cold,Cough,Heart,Masticatory,Sore,...","nan,Balsam,Canada,Balsam Fir,Fir",Pinaceae,Abies,balsamea


Exporting the transformed dataset for later use:

In [ ]:
# mod_duke_df.to_csv("../data/DUKE/PIVOT_ETHNOBOT.csv")

### Targeting only the activities to create a placeholder collection of symptoms:

In [27]:
symptoms = duke_df["ACTIVITY"].unique()
symptoms_df = pd.DataFrame(symptoms)
symptoms_df.rename(columns={0: "symptomName"}, inplace=True)
filt = (symptoms_df.symptomName.str.contains("(Veterinary)") == False)
reduced_symptoms_df = symptoms_df[filt]
reduced_symptoms_df.head()

/tmp/ipykernel_23926/433113371.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filt = (symptoms_df.symptomName.str.contains("(Veterinary)") == False)


,symptomName
0,Abcess
1,Abdomen
2,Abortifacient
4,Abortive
5,Abrasion


For placeholder json: requires ids

In [28]:
id_df = reduced_symptoms_df.copy()
reduced_symptoms_df.loc[:, "_id"] = id_df.index
reduced_symptoms_df.head()

/tmp/ipykernel_23926/1559873215.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_symptoms_df.loc[:, "_id"] = id_df.index


,symptomName,_id
0,Abcess,0
1,Abdomen,1
2,Abortifacient,2
4,Abortive,4
5,Abrasion,5


In [29]:
reduced_symptoms_df.to_json("../output/symptoms.json", orient="records")

In [31]:
check_df = pd.read_json("../output/symptoms.json")
check_df.head()

,symptomName,_id
0,Abcess,0
1,Abdomen,1
2,Abortifacient,2
3,Abortive,4
4,Abrasion,5


## Transforming the KEW dataset:

### wcvp_names.csv:

In [10]:
wcvp_names_df.shape

(1414666, 31)

As we can see, the wcvp_names_df has 1.414.666 rows of data and 31 columns. 

In [11]:
wcvp_names_df.head()

,plant_name_id,ipni_id,taxon_rank,taxon_status,family,genus_hybrid,genus,species_hybrid,species,infraspecific_rank,...,taxon_name,taxon_authors,accepted_plant_name_id,basionym_plant_name_id,replaced_synonym_author,homotypic_synonym,parent_plant_name_id,powo_id,hybrid_formula,reviewed
0,250443,309465-1,Species,Unplaced,Cyperaceae,NaN,Isolepis,NaN,panamensis,NaN,...,Isolepis panamensis,J.Presl & C.Presl,NaN,250232.0,J.Presl & C.Presl,NaN,250129.0,309465-1,NaN,Y
1,345888,77078885-1,nothosubsp.,Unplaced,Orchidaceae,NaN,Anacamptis,×,gennarii,nothosubsp.,...,Anacamptis × gennarii nothosubsp. bornemanniae,"(Asch.) H.Kretzschmar, Eccarius & H.Dietr.",NaN,142282.0,NaN,NaN,8578.0,77078885-1,A. morio subsp. longicornu × A. papilionacea,Y
2,2490178,231532-2,Species,Unplaced,Cucurbitaceae,NaN,Sechium,NaN,tacaco,NaN,...,Sechium tacaco,(Pittier) C.Jeffrey,NaN,2409709.0,NaN,NaN,2490165.0,231532-2,NaN,N
3,2602950,582307-1,Species,Synonym,Siparunaceae,NaN,Siparuna,NaN,lindenii,NaN,...,Siparuna lindenii,(Seem.) A.DC.,NaN,2723851.0,NaN,NaN,NaN,582307-1,NaN,N
4,3143990,17540160-1,Subspecies,Unplaced,Pteridaceae,NaN,Pentagramma,NaN,triangularis,subsp.,...,Pentagramma triangularis subsp. maxonii,"(Weath.) Yatsk., Windham & E.Wollenw.",NaN,3164043.0,NaN,NaN,3143986.0,17540160-1,NaN,N


Taking a look at the contents of the dataframe, the following transformations seem plausible:
* columns we care about are plant_name_id and the taxonomic name so that we can properly identify the plants from the KEW datasets
* the existing taxon_name column contains information on hybridisation and so on, so it makes sense to create a new, simplified taxon column from genus and species

#### New "taxon" column:

In [12]:
wcvp_names_df["taxon"] = wcvp_names_df["genus"] + " " + wcvp_names_df["species"].str.lower()

In [13]:
wcvp_names_df.loc[:9, "taxon"]

0         Isolepis panamensis
1         Anacamptis gennarii
2              Sechium tacaco
3           Siparuna lindenii
4    Pentagramma triangularis
5         Suregada glandulosa
6              Hosta undulata
7            Molinia caerulea
8                Rubus omalus
9              Teline pallida
Name: taxon, dtype: object

#### Reducing dataframe to only the columns we need:

In [14]:
reduced_names_df = wcvp_names_df[["plant_name_id", "taxon"]].set_index("plant_name_id")
reduced_names_df

,taxon
plant_name_id,
250443,Isolepis panamensis
345888,Anacamptis gennarii
2490178,Sechium tacaco
2602950,Siparuna lindenii
3143990,Pentagramma triangularis
...,...
3287185,Hieracium valdepilosum
3287191,Hieracium valdepilosum
3287208,Hieracium valoddae


#### Exporting a new, lightweight csv file to upload to GitHub and use for further manipulations:

In [15]:
reduced_names_df.to_csv(r"../data/WCVP/REDUCED_wcvp_names.csv")

### wcvp_distribution.csv

In [16]:
wcvp_dis_df.shape

(1950339, 11)

As we can see, the wcvp_dis_df has an astounding 1.950.339 rows of data for 11 columns. 

In [17]:
wcvp_dis_df.head()

,plant_locality_id,plant_name_id,continent_code_l1,continent,region_code_l2,region,area_code_l3,area,introduced,extinct,location_doubtful
0,1394111,1,8,SOUTHERN AMERICA,80.0,Central America,COS,Costa Rica,0,0,0
1,1394112,1,8,SOUTHERN AMERICA,82.0,Northern South America,VEN,Venezuela,0,0,0
2,1394113,1,8,SOUTHERN AMERICA,83.0,Western South America,BOL,Bolivia,0,0,0
3,1394114,1,8,SOUTHERN AMERICA,83.0,Western South America,CLM,Colombia,0,0,0
4,1394115,1,8,SOUTHERN AMERICA,83.0,Western South America,ECU,Ecuador,0,0,0


Taking a look at the contents of the dataframe, the following transformations seem plausible:
* columns we care about are plant_name_id (for matching to the names dataframe) and continent, region, area (although not all three are necessary)
* instead of unique pairings of plant_locality_id and plant_name_id, we can pivot the dataframe by unique plant_name_id instead

#### Reducing dataframe to only the columns we need:

In [18]:
reduced_dis_df = wcvp_dis_df[["plant_name_id", "continent", "region", "area"]]
reduced_dis_df

,plant_name_id,continent,region,area
0,1,SOUTHERN AMERICA,Central America,Costa Rica
1,1,SOUTHERN AMERICA,Northern South America,Venezuela
2,1,SOUTHERN AMERICA,Western South America,Bolivia
3,1,SOUTHERN AMERICA,Western South America,Colombia
4,1,SOUTHERN AMERICA,Western South America,Ecuador
...,...,...,...,...
1950334,3287516,AFRICA,West-Central Tropical Africa,Cameroon
1950335,3287542,ASIA-TROPICAL,Malesia,Borneo
1950336,3287543,ASIA-TROPICAL,Malesia,Borneo
1950337,3287544,EUROPE,Northern Europe,Finland


In testing, this has already brought the file size from 138MB down to 106MB.

#### Pivoting:

In [19]:
pivot_dis_df = reduced_dis_df.pivot_table(index='plant_name_id', aggfunc=pd.unique)
pivot_dis_df

,area,continent,region
plant_name_id,,,
1,"[Costa Rica, Venezuela, Bolivia, Colombia, Ecu...",[SOUTHERN AMERICA],"[Central America, Northern South America, West..."
2,"[Argentina Northeast, Argentina Northwest]",[SOUTHERN AMERICA],[Southern South America]
3,"[Colombia, Ecuador, Peru, Brazil North]",[SOUTHERN AMERICA],"[Western South America, Brazil]"
5,"[Bolivia, Peru]",[SOUTHERN AMERICA],[Western South America]
7,"[Colombia, Ecuador]",[SOUTHERN AMERICA],[Western South America]
...,...,...,...
3287516,[Cameroon],[AFRICA],[West-Central Tropical Africa]
3287542,[Borneo],[ASIA-TROPICAL],[Malesia]
3287543,[Borneo],[ASIA-TROPICAL],[Malesia]


1,950,339 rows down to 435,009 rows.

In testing, this brought the file size further down to 58MB (which is still too large for default GitHub uploads). Upon further consideration, the region column in particular seems the least useful and will therefore also be excluded in the final transformation:

In [20]:
reduced_dis_df = wcvp_dis_df[["plant_name_id", "continent", "area"]]
pivot_dis_df = reduced_dis_df.pivot_table(index='plant_name_id', aggfunc=pd.unique)
pivot_dis_df

,area,continent
plant_name_id,,
1,"[Costa Rica, Venezuela, Bolivia, Colombia, Ecu...",[SOUTHERN AMERICA]
2,"[Argentina Northeast, Argentina Northwest]",[SOUTHERN AMERICA]
3,"[Colombia, Ecuador, Peru, Brazil North]",[SOUTHERN AMERICA]
5,"[Bolivia, Peru]",[SOUTHERN AMERICA]
7,"[Colombia, Ecuador]",[SOUTHERN AMERICA]
...,...,...
3287516,[Cameroon],[AFRICA]
3287542,[Borneo],[ASIA-TROPICAL]
3287543,[Borneo],[ASIA-TROPICAL]


In [21]:
# pivot_dis_df.to_csv(r"../data/WCVP/REDUCED_PIVOT_wcvp_distribution.csv")

##### Alternative to storing an array: 

Instead of aggregating values in a list (which causes issues with how the csv will be read later on), save it in a comma separated string. 

This throws an error and causes the area column to be dropped:

_['area'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning._

In [22]:
agg_func = lambda x: ",".join(x.unique())

In [23]:
# pivot_dis_df = reduced_dis_df.pivot_table(index='plant_name_id', aggfunc=agg_func)
# pivot_dis_df

##### Alternative 2 to storing an array: 

Transform values saved in pivot_dis_df after the fact:

In [24]:
pivot_dis_df_copy = pivot_dis_df.copy()
pivot_dis_df_copy["continent"] = pivot_dis_df_copy["continent"].str.join(",")
pivot_dis_df_copy["area"] = pivot_dis_df_copy["area"].str.join(",")
pivot_dis_df_copy

,area,continent
plant_name_id,,
1,"Costa Rica,Venezuela,Bolivia,Colombia,Ecuador,...",SOUTHERN AMERICA
2,"Argentina Northeast,Argentina Northwest",SOUTHERN AMERICA
3,"Colombia,Ecuador,Peru,Brazil North",SOUTHERN AMERICA
5,"Bolivia,Peru",SOUTHERN AMERICA
7,"Colombia,Ecuador",SOUTHERN AMERICA
...,...,...
3287516,Cameroon,AFRICA
3287542,Borneo,ASIA-TROPICAL
3287543,Borneo,ASIA-TROPICAL


In [25]:
# pivot_dis_df_copy.to_csv(r"../data/WCVP/STR_REDUCED_PIVOT_wcvp_distribution.csv")

This actually further reduced the file size to 31MB.

#### Comparing output of these new csv files:

In [26]:
dis_df_a = pd.read_csv("../data/WCVP/REDUCED_PIVOT_wcvp_distribution.csv", sep=",")
dis_df_b = pd.read_csv("../data/WCVP/STR_REDUCED_PIVOT_wcvp_distribution.csv", sep=",")

Looking at the dataframes, the one generated from the initial approach seems to store the area in a [] list with each individual string surrounded by single quotation marks (spoiler: it's actually not a list/array).

The alternative approach of transforming entries from a list into a joined string stores the values separated by commas, without any brackets and quotation marks appearing at all.

In [27]:
dis_df_a.head()

,plant_name_id,area,continent
0,1,['Costa Rica' 'Venezuela' 'Bolivia' 'Colombia'...,['SOUTHERN AMERICA']
1,2,['Argentina Northeast' 'Argentina Northwest'],['SOUTHERN AMERICA']
2,3,['Colombia' 'Ecuador' 'Peru' 'Brazil North'],['SOUTHERN AMERICA']
3,5,['Bolivia' 'Peru'],['SOUTHERN AMERICA']
4,7,['Colombia' 'Ecuador'],['SOUTHERN AMERICA']


In [28]:
dis_df_b.head()

,plant_name_id,area,continent
0,1,"Costa Rica,Venezuela,Bolivia,Colombia,Ecuador,...",SOUTHERN AMERICA
1,2,"Argentina Northeast,Argentina Northwest",SOUTHERN AMERICA
2,3,"Colombia,Ecuador,Peru,Brazil North",SOUTHERN AMERICA
3,5,"Bolivia,Peru",SOUTHERN AMERICA
4,7,"Colombia,Ecuador",SOUTHERN AMERICA


When we take a look at an individual entry, however, we can see why the alternative approach is a lot more useable. The "list" of the initial approach is not a list at all, but rather a string containing brackets and quotation marks. __Extracting individual entries in a useable way therefore requires a lot of extra steps.__

In [29]:
print(type(dis_df_a.loc[0, "area"]))

# trying to store individual entries in an actual python list:
a_values = dis_df_a.loc[0, "area"]
a_values = a_values.split("'")

# splitting like this leaves nonsensical entries in the list, such as '[', ' ', 
# and even '\n '
bad_entries = ["[", "]", " ", "\n "]
for entry in bad_entries:
# recursive, since " " appears several times
    while entry in a_values:
        a_values.remove(entry)

print(a_values)
print(type(a_values))

<class 'str'>
['Costa Rica', 'Venezuela', 'Bolivia', 'Colombia', 'Ecuador', 'Peru', 'Brazil North', 'Argentina Northeast', 'Argentina Northwest']
<class 'list'>


Compared to extracting a useable list from the alternative csv file:

In [30]:
print(type(dis_df_b.loc[0, "area"]))

b_values = dis_df_b.loc[0, "area"]
b_values = b_values.split(",")

print(b_values)
print(type(b_values))

<class 'str'>
['Costa Rica', 'Venezuela', 'Bolivia', 'Colombia', 'Ecuador', 'Peru', 'Brazil North', 'Argentina Northeast', 'Argentina Northwest']
<class 'list'>


## Extracting useable data from the WHO monograph:

### Reading the Monograph and generating lists, as per the data exploration notebook:

In [54]:
duke_names = duke_df["TAXON"].unique().tolist()

In [55]:
pattern_formula = re.compile(r"\.[A-Za-z0-9]+.*")

# Definitions with in-keyword:

who_01_def = list()
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Definition" in line:
            n_lines = 1
            while page[i + n_lines] != "Synonyms" and i + n_lines < max_len:
                n_lines += 1
            who_01_def.append(" ".join(page[i + 1 : i + n_lines]))

# Vernacular names with in-keyword:

who_01_vernacular = list()
reg_pattern = re.compile(r"\.")
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Selected vernacular names" in line:
            n_lines = 1
            while page[i + n_lines] != "Description" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
            else:
                output = " ".join(page[i + 1 : i + n_lines])
            
            output =  output.replace(".", ",")
            
            # Sometimes taxonomic names appear in the WHO document names as a header of sorts:
            for name in duke_names:
                output = output.replace(name, ",")
            
            output =  output.replace(", ", ",")

            who_01_vernacular.append(output)

# Clinical medicine with in-keyword:

who_01_clinical = list()
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Uses supported by clinical data" in line:
            n_lines = 1
            while "Uses described in pharmacopoeias and in traditional" not in page[i + n_lines] and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            # in case the description gets cut off, drop the incomplete sentence:
            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."
            
            who_01_clinical.append(output)

# Traditional medicine with in-keyword:

who_01_traditional = list()
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Uses described in pharmacopoeias and in traditional" in line:
            n_lines = 2
            while "Uses described in folk medicine" not in page[i + n_lines] and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 2 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 2 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_traditional.append(output)

# Folk with in-keyword:

who_01_folk = list()
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Uses described in folk medicine" in line:
            n_lines = 2
            while page[i + n_lines] != "Pharmacology" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 2 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 2 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_folk.append(output)

who_01_con = list()
  
for page in who_01_pages_corrected[4:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Contraindications" in line and "see" not in line.lower():
            n_lines = 1
            while page[i + n_lines] != "Warnings" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_con.append(output)
            
who_01_warn = list()
  
for page in who_01_pages_corrected[4:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Warnings" in line and "see" not in line.lower():
            n_lines = 1
            while page[i + n_lines] != "Precautions" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_warn.append(output)
            
who_01_adv = list()

ref_pattern = re.compile(r"[0-9]+\.")  

for page in who_01_pages_corrected[4:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Adverse reactions" in line and "see" not in line.lower() and not re.search(ref_pattern, line):
            n_lines = 1
            while page[i + n_lines] != "Posology" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_adv.append(output)

who_01_pos = list()
  
for page in who_01_pages_corrected[4:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Posology" in line and "and" not in line.lower():
            n_lines = 1
            while page[i + n_lines] != "References" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                # output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                # output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_pos.append(output)

Replace WHO null values with empty string:

In [56]:
null_values = ["No information available.", "None."]

def make_nan(target_list: list):
    for i, value in enumerate(target_list):
        if value in null_values:
            target_list[i] = ""

In [57]:
make_nan(who_01_clinical)
make_nan(who_01_traditional)
make_nan(who_01_folk)
make_nan(who_01_con)
make_nan(who_01_warn)
make_nan(who_01_adv)
make_nan(who_01_pos)

## Cross-reference DUKE:

It is possible to compare entries to DUKE dataset. When it is a match, we append a dictionary with name, definition etc. keys for the matched herb to a list.

The idea is to eventually add columns to the DUKE dataframe that includes some of this information.

### List of dictionaries:

In [58]:
who_01_remedies = list()

for i, definition in enumerate(who_01_def):

    remedy_dict = dict()

    for name in duke_names:
        if name in definition:
            remedy_dict["NAME"] = name
            remedy_dict["DEF"] = definition
            remedy_dict["VERNAC"] = who_01_vernacular[i]
            remedy_dict["CLINICAL"] = who_01_clinical[i]
            remedy_dict["TRADITIONAL"] = who_01_traditional[i]
            remedy_dict["FOLK"] = who_01_folk[i]

    if remedy_dict:
        who_01_remedies.append(remedy_dict)


#### Missing herbs:

We know that all the lists now have a length of 28 entries, however the dictionary created only contains 26:

In [71]:
print("Length of a list:", len(who_01_def), "\nLength of the dictionary:", len(who_01_remedies))

Length of a list: 28 
Length of the dictionary: 26


We will run a quick algorithm to see which two herbs were not included because their names did not return a match with the current dictionary generation code:

In [63]:
matches = list()

for definition in who_01_def:
    for herb in who_01_remedies:
        if definition in herb["DEF"]:
            matches.append(definition)

rest = list()

for definition in who_01_def:
    if definition not in matches:
        rest.append(definition)

In [68]:
print("Length of matches:", len(matches), "\nLength of rest:", len(rest), end="\n\n")
print(*rest, sep="\n")

Length of matches: 26 
Length of rest: 2

Flos Chamomillae consists of the dried flowering heads of Chamomilla recutita (L.) Rauschert (Asteraceae).
Semen Plantaginis is the dried, ripe seed of Plantago afra L., P. indica L., P. ovata Forsk., or P. asiatica L. (Plantaginaceae).


Excluded herbs were:
* Chamomilla recutita
* Plantago afra

Filtering the Duke dataframe for "Chamomilla":

In [69]:
filt = (duke_df["TAXON"].str.contains("Chamomilla"))
duke_df[filt]["TAXON"]

Series([], Name: TAXON, dtype: object)

Trying the lowercase "chamomilla" instead:

In [70]:
filt = (duke_df["TAXON"].str.contains("chamomilla"))
duke_df[filt]["TAXON"].unique()

array(['Matricaria chamomilla'], dtype=object)

From __Wikipedia, The Free Encyclopedia__: "Matricaria chamomilla (synonym: Matricaria recutita), commonly known as chamomile"

The WHO monograph and Duke's Phytochemical and Ethnobotanical Database refer to the same herb by different names.

Filtering the Duke dataframe for "Plantago afra":

In [72]:
filt = (duke_df["TAXON"].str.contains("Plantago afra"))
duke_df[filt]["TAXON"].unique()

array([], dtype=object)

Trying just "Plantago" instead to find similar matches:

In [73]:
filt = (duke_df["TAXON"].str.contains("Plantago"))
duke_df[filt]["TAXON"].unique()

array(['Plantago asiatica', 'Plantago major', 'Plantago erosa',
       'Plantago insularis', 'Plantago lanceolata', 'Plantago ovata',
       'Plantago majorvar.asiatica', 'Plantago psyllium',
       'Plantago media', 'Plantago sp', 'Plantago cordata',
       'Plantago depressa', 'Plantago coronopus',
       'Plantago amplexicaulis', 'Plantago cynops', 'Plantago rugelii',
       'Plantago griesebachii', 'Plantago lagopus', 'Plantago rocae',
       'Plantago sericea', 'Plantago ureades', 'Plantago ispagula',
       'Plantago macrostachys', 'Plantago minor', 'Plantago ciliata',
       'Plantago mexicana', 'Plantago tomentosa', 'Plantago coronopus?'],
      dtype=object)

From its definition, we know that "Plantago afra" has the following synonyms: 

"_Plantago afra L., P. indica L., P. ovata Forsk., or P. asiatica_"

As we can see in the above array returned by the Duke dataframe, both _Plantago ovata_ and _Plantago asiatica_ are in fact part of the Duke dataset.

#### Conclusion:

Both _Chamomilla recutita_ and _Plantago afra_ will have to have their entries corrected manually.

For _Plantago afra_, maybe a solution when initially checking for the name: check if the split name is in the definition.

In [74]:
matches = list()

for definition in who_01_def:
    for name in duke_names:
        name = name.split(" ")

        # some taxonomic names have an "x", indicating hybrids
        if "x" in name: 
            name.remove("x")

        if name[0] in definition and name[1] in definition:
            matches.append(" ".join(name))

print(*set(matches), sep=" - ")
len(matches)

Allium sativum - Platycodon grandiflorum - Aloe vera - Rheum palmatum - Coptis chinensis - Panax ginseng - Thymus vulgaris - Zingiber officinale - Echinacea pallida - Thymus zygis - Ginkgo biloba - Coptis sp - Valeriana officinalis - Echinacea purpurea - Aloe africana - Curcuma longa - Echinacea angustifolia - Paeonia lactiflora - Cassia senna - Aloe sp - Brucea javanica - Glycyrrhiza uralensis - Zingiber sp - Plantago ovata - Plantago asiatica - Glycyrrhiza glabra - Allium cepa - Aloe spicata - Astragalus mongholicus - Ephedra sinica - Glycyrrhiza ralensis - Cinnamomum cassia - Amomum sp - Cinnamomum verum - Coptis japonica - Bupleurum falcatum - Astragalus membranaceus - Rheum officinale - Centella asiatica - Ephedra sp - Rauvolfia serpentina - Aloe ferox


44

What is especially interesting to see is that we now actually have 44 matches.

The definition often includes synonyms (such as we saw with _Plantago Afra_) so the subsequent information is relevant for a number of herbs in the Duke dataset. 

In [75]:
for match in matches:
    if "Plantago" in match:
        print(match)

Plantago asiatica
Plantago ovata


But still no matches for "Chamomille", as its definition did not include a synonym that matched with the Duke dataset:

In [76]:
for match in matches:
    if "chamomilla" in match:
        print(match)

### Final dictionary generation code:

In [79]:
who_01_remedies = list()

for i, definition in enumerate(who_01_def):

    for name in duke_names:

        remedy_dict = dict()
        name = name.split(" ")

        # some taxonomic names have an "x", indicating hybrids
        if "x" in name: 
            name.remove("x")

        if name[0] in definition and name[1] in definition:
            remedy_dict["NAME"] = " ".join(name)
            remedy_dict["DEF"] = definition
            remedy_dict["VERNAC"] = who_01_vernacular[i]
            remedy_dict["CLINICAL"] = who_01_clinical[i]
            remedy_dict["TRADITIONAL"] = who_01_traditional[i] 
            remedy_dict["FOLK"] = who_01_folk[i]
            remedy_dict["CONTRAINDICATION"] = who_01_con[i]
            remedy_dict["WARNING"] = who_01_warn[i]
            remedy_dict["ADVERSE"] = who_01_adv[i]
            remedy_dict["POSOLOGY"] = who_01_pos[i]

        if remedy_dict:
            who_01_remedies.append(remedy_dict)

len(who_01_remedies)

44

In [78]:
print(who_01_remedies[0])

{'NAME': 'Allium cepa', 'DEF': 'Bulbus Allii Cepae is the fresh or dried bulbs of Allium cepa L. (Liliaceae) or its varieties and cultivars.', 'VERNAC': 'It is most commonly known as “onion”,Basal,basl,cebolla,cebolla morada,cepa bulb,cepolla,cipolla,common onion,cu hanh,hom hua yai,hom khaao,hom yai,hu-t’sung,hu t’sung t’song,hua phak bhu,i-i-bsel,kesounni,khtim,Küchenzwiebel,l’oignon,loyon,Madras oignon,oignon,palandu,piyaj,piyaz,pyaz,pyaaz,ralu lunu,red globe onion,sibuyas,Spanish onion,tamanegi,umbi bawang merah,vengayan,yellow Bermuda onion,white globe onion,Zwiebel ,', 'CLINICAL': 'The principal use of Bulbus Allii Cepae today is to prevent age-dependent changes in the blood vessels, and loss of appetite.', 'TRADITIONAL': 'Treatment of bacterial infections such as dysentery, and as a diuretic. The drug has also been used to treat ulcers, wounds, scars, keloids, and asthma . Bulbus Allii Cepae has also been used as an adjuvant therapy for diabetes.', 'FOLK': 'As an anthelminthic, 

This code will be used in the next notebook as a basis to enrich the Duke dataframe.

Output name of herbs

In [24]:
mod_duke_df["popularityRating"] = np.nan

In [56]:
mod_duke_df.rename(columns={
    "TAXON": "taxonomicName",
    "ACTIVITY": "activity",
    "CNAME": "commonNames"
    }, inplace=True)
mod_duke_df

,taxonomicName,activity,commonNames,FAMILY,GENUS,SPECIES,popularityRating
0,Abelmoschus esculentus,"Abortifacient,Antidote,Boil,Burn,Catarrh,Coffe...","nan,Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,...",Malvaceae,Abelmoschus,esculentus,NaN
1,Abelmoschus manihot,"Antitussive,Boil,Cancer,Cancer(Stomach),Catarr...","Tororo-Aoi,nan,Kastuli",Malvaceae,Abelmoschus,manihot,NaN
2,Abelmoschus moschatus,"Aphrodisiac,Asthma,Bite(Snake),Boil,Cancer,Car...","Musk Mallow,nan,Moskus,Muskus,Mushk Dana,Amber...",Malvaceae,Abelmoschus,moschatus,NaN
3,Abies alba,"Bronchitis,Bruise,Calculus,Catarrh,Cough,Diure...","Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ...",Pinaceae,Abies,alba,NaN
4,Abies balsamea,"Burn,Cancer,Cold,Cough,Heart,Masticatory,Sore,...","nan,Balsam,Canada,Balsam Fir,Fir",Pinaceae,Abies,balsamea,NaN
...,...,...,...,...,...,...,...
13074,Zostera marina,Diarrhea,nan,Zosteraceae,Zostera,marina,NaN
13075,Zuelania guidonia,Diuretic,Guuaguasi,Flacourtiaceae,Zuelania,guidonia,NaN
13076,Zuelania roussoviana,Emetic,nan,Flacourtiaceae,Zuelania,roussoviana,NaN
13077,Zygophyllum fabago,"Spasm,Vermifuge,Depurative,Poison","Khennaiq Ad Dijaj,Morsana",Zygophyllaceae,Zygophyllum,fabago,NaN


In [57]:
taxonomy_df = mod_duke_df[["taxonomicName", "popularityRating", "commonNames"]]
taxonomy_df.set_index("taxonomicName", inplace=True)
taxonomy_df.head()

,popularityRating,commonNames
taxonomicName,,
Abelmoschus esculentus,NaN,"nan,Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,..."
Abelmoschus manihot,NaN,"Tororo-Aoi,nan,Kastuli"
Abelmoschus moschatus,NaN,"Musk Mallow,nan,Moskus,Muskus,Mushk Dana,Amber..."
Abies alba,NaN,"Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ..."
Abies balsamea,NaN,"nan,Balsam,Canada,Balsam Fir,Fir"


In [58]:
taxonomy_df.commonNames = taxonomy_df.commonNames.str.replace('nan,', '', regex=True)

/tmp/ipykernel_8021/3636771354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxonomy_df.commonNames = taxonomy_df.commonNames.str.replace('nan,', '', regex=True)


In [59]:
nan_toreplace = ["nan", ""]
taxonomy_df = taxonomy_df.replace(nan_toreplace, np.nan)

In [60]:
taxonomy_df.dropna(subset="commonNames")

,popularityRating,commonNames
taxonomicName,,
Abelmoschus esculentus,NaN,"Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,Gomb..."
Abelmoschus manihot,NaN,"Tororo-Aoi,Kastuli"
Abelmoschus moschatus,NaN,"Musk Mallow,Moskus,Muskus,Mushk Dana,Ambercice..."
Abies alba,NaN,"Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ..."
Abies balsamea,NaN,"Balsam,Canada,Balsam Fir,Fir"
...,...,...
Ziziphus vulgaris,NaN,"Tsao,Mei Tsao,Kan Tsao,Liang Tsao,Ta Tsao,Pei ..."
Ziziphus vulgarisvar.inermis,NaN,Sanebuto-Natume
Zizyphus vulgaris,NaN,Tsao


In [50]:
filter = taxonomy_df.commonNames.str.contains('Ginger')
filt_df = taxonomy_df[filter]
filt_df

,popularityRating,commonNames
taxonomicName,,
Asarum canadense,NaN,"Kanada Azarumu,Wild Ginger"
Parinarium macrophyllum,NaN,"Plum,Gingerbread,Gingerbread Plum"
Zingiber cassumunar,NaN,"Bengle,Bangle,Bunglai,Kunyit bolai,Panini,Oeni..."
Zingiber mioga,NaN,"Jang Ho,Mioga Ginger"
Zingiber officinale,NaN,"Chiang,Gingembre,Sheng Chiang,Ginger,Gan Jiang..."
Zingiber zerumbet,NaN,"Lampuyang gajah,Lampuyang kebo,Lampuyang paek,..."


In [43]:
taxonomy_df.head()

,popularityRating,commonNames
taxonomicName,,
Abelmoschus esculentus,NaN,"Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,Gomb..."
Abelmoschus manihot,NaN,"Tororo-Aoi,Kastuli"
Abelmoschus moschatus,NaN,"Musk Mallow,Moskus,Muskus,Mushk Dana,Ambercice..."
Abies alba,NaN,"Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ..."
Abies balsamea,NaN,"Balsam,Canada,Balsam Fir,Fir"


In [44]:
taxonomy_df.to_csv("../output/taxonomic_names.csv", sep=",")

In [45]:
test_df = pd.read_csv("../output/taxonomic_names.csv")

In [46]:
test_df

,taxonomicName,popularityRating,commonNames
0,Abelmoschus esculentus,NaN,"Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,Gomb..."
1,Abelmoschus manihot,NaN,"Tororo-Aoi,Kastuli"
2,Abelmoschus moschatus,NaN,"Musk Mallow,Moskus,Muskus,Mushk Dana,Ambercice..."
3,Abies alba,NaN,"Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ..."
4,Abies balsamea,NaN,"Balsam,Canada,Balsam Fir,Fir"
...,...,...,...
13074,Zostera marina,NaN,NaN
13075,Zuelania guidonia,NaN,Guuaguasi
13076,Zuelania roussoviana,NaN,NaN
13077,Zygophyllum fabago,NaN,"Khennaiq Ad Dijaj,Morsana"


## Looking Ahead:

The next few notebooks will explore a different avenue to supplement our data: using the pytrends module to pull popularity data for herbs from Google Trends, in the hopes of using that data as a general popularity indicator of a given herb. For now, we will not worry about getting region-specific data.